Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 8.30.0

torch: 2.6.0+cu126



# Filter Response Normalization for Network-in-Network CIFAR-10 Classifier

The CNN architecture is based on  
CNN架构是基于  

- Lin, Min, Qiang Chen, and Shuicheng Yan. "[Network in network](https://arxiv.org/abs/1312.4400)." arXiv preprint arXiv:1312.4400 (2013).  
- 林敏，陈强，颜水成。"[网络中的网络](https://arxiv.org/abs/1312.4400)"。arXiv预印本 arXiv:1312.4400（2013年）。  

This notebook implements Filter Response Normalization as a drop-in replacement for BatchNorm, based on the paper:  
本笔记本实现了过滤响应归一化，作为BatchNorm的替代方案，基于以下论文：  

- S. Singh and S. Krishnan (2019). **Filter Response Normalization Layer: Eliminating Batch Dependence in the Training of Deep Neural Networks.** https://arxiv.org/abs/1911.09737  
- S. Singh 和 S. Krishnan（2019年）。**过滤响应归一化层：消除深度神经网络训练中的批次依赖性。** https://arxiv.org/abs/1911.09737

In [2]:
import torch
import torch.nn as nn


class FilterResponseNormalization(nn.Module):
    def __init__(self, num_features, eps=1e-6):
        super(FilterResponseNormalization, self).__init__()
        
        # 注册可训练参数beta，初始化为服从正态分布的张量
        self.register_parameter('beta', 
                                torch.nn.Parameter(
                                        torch.empty([1, num_features, 1, 1]).normal_()))
    
        # 注册可训练参数gamma，初始化为服从正态分布的张量
        self.register_parameter('gamma', 
                                torch.nn.Parameter(
                                        torch.empty([1, num_features, 1, 1]).normal_()))
        
        # 注册可训练参数tau，初始化为服从正态分布的张量
        self.register_parameter('tau', 
                                torch.nn.Parameter(
                                        torch.empty([1, num_features, 1, 1]).normal_()))
        
        self.eps = torch.Tensor([eps])  # 初始化eps，防止除零错误

    def forward(self, x):
        # 前向传播函数，参考了
        # https://github.com/gupta-abhay/pytorch-frn/blob/master/frn.py
        n, c, h, w = x.size()  # 获取输入张量的尺寸
        
        self.eps = self.eps.to(self.tau.device)  # 确保eps在与tau相同的设备上

        # 计算输入张量x的平方平均值（通道维度上的均值）
        nu2 = torch.mean(x.pow(2), (2, 3), keepdims=True)
        
        # 对输入张量x进行标准化，利用平方均值和eps
        x = x * torch.rsqrt(nu2 + torch.abs(self.eps))
        
        # 返回经过FRN归一化处理后的结果，包含gamma、beta和tau的影响
        return torch.max(self.gamma * x + self.beta, self.tau)


## Additional Imports

In [3]:
import os
import time

import numpy as np
import pandas as pd

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.cuda.set_per_process_memory_fraction(0.5, device=0)

## Model Settings

In [4]:
##########################
### 设置
##########################

# 超参数
RANDOM_SEED = 1
LEARNING_RATE = 0.00005
BATCH_SIZE = 256
NUM_EPOCHS = 100

# 网络架构
NUM_CLASSES = 10

# 其他设置
DEVICE = "cuda:0"
GRAYSCALE = False

In [5]:
##########################
### CIFAR-10 数据集
##########################

# 注意 transforms.ToTensor() 会将输入图像缩放到 0-1 范围内

train_indices = torch.arange(0, 49000)  # 训练集的索引
valid_indices = torch.arange(49000, 50000)  # 验证集的索引

# 加载 CIFAR-10 数据集，train=True 表示训练集，transform=transforms.ToTensor() 将图像转为 Tensor 格式，下载数据集
train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=transforms.ToTensor(),
                                   download=True)

# 从 CIFAR-10 数据集中提取训练集和验证集
train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

# 加载测试集，train=False 表示测试集，transform=transforms.ToTensor() 将图像转为 Tensor 格式
test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())

#####################################################
### 数据加载器
#####################################################

# 创建训练数据加载器，batch_size 为批次大小，num_workers 设置为 8 表示使用 8 个子进程加载数据，shuffle=True 表示打乱数据
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=True)

# 创建验证数据加载器，shuffle=False 表示不打乱验证集数据
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=8,
                          shuffle=False)

# 创建测试数据加载器，shuffle=False 表示不打乱测试集数据
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

#####################################################

# 检查数据集的形状
for images, labels in train_loader:  
    print('训练集图像批次的维度:', images.shape)  # 输出训练集图像的维度
    print('训练集标签批次的维度:', labels.shape)  # 输出训练集标签的维度
    break

for images, labels in test_loader:  
    print('测试集图像批次的维度:', images.shape)  # 输出测试集图像的维度
    print('测试集标签批次的维度:', labels.shape)  # 输出测试集标签的维度
    break
    
for images, labels in valid_loader:  
    print('验证集图像批次的维度:', images.shape)  # 输出验证集图像的维度
    print('验证集标签批次的维度:', labels.shape)  # 输出验证集标签的维度
    break

训练集图像批次的维度: torch.Size([256, 3, 32, 32])
训练集标签批次的维度: torch.Size([256])
测试集图像批次的维度: torch.Size([256, 3, 32, 32])
测试集标签批次的维度: torch.Size([256])
验证集图像批次的维度: torch.Size([256, 3, 32, 32])
验证集标签批次的维度: torch.Size([256])


# Filter Response Normalization

In [6]:
##########################
### 模型
##########################

class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        # 定义网络的分类器部分
        self.classifier = nn.Sequential(
                # 第一层卷积层：输入通道为 3，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）进行标准化
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第二层卷积层：输入通道为 192，输出通道为 160，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0, bias=False),
                FilterResponseNormalization(160),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第三层卷积层：输入通道为 160，输出通道为 96，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0, bias=False),
                FilterResponseNormalization(96),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 最大池化层：卷积后的输出通过最大池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第四层卷积层：输入通道为 96，输出通道为 192，卷积核大小为 5，步幅为 1，padding 为 2，bias=False 不使用偏置
                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第五层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第六层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 3，步幅为 2，padding 为 1
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                
                # Dropout 层：在训练时随机丢弃部分神经元，防止过拟合
                nn.Dropout(0.5),

                # 第七层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 3，步幅为 1，padding 为 1，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第八层卷积层：输入通道为 192，输出通道为 192，卷积核大小为 1，步幅为 1，padding 为 0，bias=False 不使用偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                FilterResponseNormalization(192),  # 使用滤波响应归一化（FRN）
                #nn.ReLU(inplace=True),  # ReLU 激活函数已注释，不使用激活函数
                
                # 第九层卷积层：输入通道为 192，输出通道为 10，卷积核大小为 1，步幅为 1，padding 为 0
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),  # 激活函数 ReLU
                
                # 平均池化层：卷积后的输出通过平均池化层进行下采样，卷积核大小为 8，步幅为 1，padding 为 0
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                )

    def forward(self, x):
        # 前向传播过程
        x = self.classifier(x)  # 通过分类器进行计算
        logits = x.view(x.size(0), self.num_classes)  # 将输出展平，形状为 [batch_size, num_classes]
        probas = torch.softmax(logits, dim=1)  # 使用 softmax 激活函数获得每个类的概率分布
        return logits, probas  # 返回 logits 和概率

In [7]:
torch.manual_seed(RANDOM_SEED)

model = NiN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [8]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)  # 将特征数据移到指定设备
        targets = targets.to(device)    # 将目标数据移到指定设备

        logits, probas = model(features)  # 获取模型输出的logits和概率
        _, predicted_labels = torch.max(probas, 1)  # 取最大概率的类别作为预测标签
        num_examples += targets.size(0)  # 累加样本数量
        correct_pred += (predicted_labels == targets).sum()  # 累加预测正确的样本数
    return correct_pred.float()/num_examples * 100  # 返回正确预测率百分比
    

start_time = time.time()  # 记录训练开始时间
for epoch in range(NUM_EPOCHS):  # 遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 遍历每个batch
    
        ### 准备每个小批量数据
        features = features.to(DEVICE)  # 将特征数据移到指定设备
        targets = targets.to(DEVICE)    # 将目标数据移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型输出的logits和概率
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清除旧的梯度
        
        cost.backward()  # 计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 根据梯度更新模型参数
        
        ### 日志输出
        if not batch_idx % 120:  # 每隔120个batch打印一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')  # 输出当前epoch和batch的损失值

    # 在计算准确率时不需要构建计算图以节省内存
    with torch.set_grad_enabled(False):
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集的准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集的准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出训练集和验证集的准确率
        
    elapsed = (time.time() - start_time)/60  # 计算训练已经进行的时间
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出经过的时间
  
elapsed = (time.time() - start_time)/60  # 计算总的训练时间
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总的训练时间

Epoch: 001/100 | Batch 000/192 | Cost: 2.3031
Epoch: 001/100 | Batch 120/192 | Cost: 2.2107
Epoch: 001/100 Train Acc.: 23.16% | Validation Acc.: 26.60%
Time elapsed: 1.82 min
Epoch: 002/100 | Batch 000/192 | Cost: 2.1562
Epoch: 002/100 | Batch 120/192 | Cost: 2.0779
Epoch: 002/100 Train Acc.: 24.90% | Validation Acc.: 28.30%
Time elapsed: 3.63 min
Epoch: 003/100 | Batch 000/192 | Cost: 2.1025
Epoch: 003/100 | Batch 120/192 | Cost: 2.0373
Epoch: 003/100 Train Acc.: 29.09% | Validation Acc.: 32.20%
Time elapsed: 4.83 min
Epoch: 004/100 | Batch 000/192 | Cost: 1.9805
Epoch: 004/100 | Batch 120/192 | Cost: 1.9233
Epoch: 004/100 Train Acc.: 31.22% | Validation Acc.: 32.90%
Time elapsed: 5.56 min
Epoch: 005/100 | Batch 000/192 | Cost: 1.9335
Epoch: 005/100 | Batch 120/192 | Cost: 1.8953
Epoch: 005/100 Train Acc.: 35.07% | Validation Acc.: 36.00%
Time elapsed: 6.28 min
Epoch: 006/100 | Batch 000/192 | Cost: 1.8593
Epoch: 006/100 | Batch 120/192 | Cost: 1.8514
Epoch: 006/100 Train Acc.: 37.11%

In [9]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 63.69%


## Batch Normalization (for comparison)

In [10]:
##########################
### 模型
##########################

class NiN(nn.Module):
    def __init__(self, num_classes):
        super(NiN, self).__init__()
        self.num_classes = num_classes
        self.classifier = nn.Sequential(
                # 第一层卷积层，输入3通道，输出192通道，卷积核大小为5，步幅为1，padding为2，且没有偏置
                nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2, bias=False),
                # 对192通道进行BatchNorm，标准化处理
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第二层卷积层，输入192通道，输出160通道，卷积核大小为1，步幅为1，padding为0，且没有偏置
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0, bias=False),
                # 对160通道进行BatchNorm
                nn.BatchNorm2d(160),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第三层卷积层，输入160通道，输出96通道，卷积核大小为1，步幅为1，padding为0，且没有偏置
                nn.Conv2d(160, 96, kernel_size=1, stride=1, padding=0, bias=False),
                # 对96通道进行BatchNorm
                nn.BatchNorm2d(96),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                # 最大池化层，池化核大小为3，步幅为2，padding为1
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                # Dropout层，丢弃概率为0.5
                nn.Dropout(0.5),

                # 第四层卷积层，输入96通道，输出192通道，卷积核大小为5，步幅为1，padding为2，且没有偏置
                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2, bias=False),
                # 对192通道进行BatchNorm
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第五层卷积层，输入192通道，输出192通道，卷积核大小为1，步幅为1，padding为0，且没有偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                # 对192通道进行BatchNorm
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第六层卷积层，输入192通道，输出192通道，卷积核大小为1，步幅为1，padding为0，且没有偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                # 对192通道进行BatchNorm
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                # 平均池化层，池化核大小为3，步幅为2，padding为1
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                # Dropout层，丢弃概率为0.5
                nn.Dropout(0.5),

                # 第七层卷积层，输入192通道，输出192通道，卷积核大小为3，步幅为1，padding为1，且没有偏置
                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1, bias=False),
                # 对192通道进行BatchNorm
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第八层卷积层，输入192通道，输出192通道，卷积核大小为1，步幅为1，padding为0，且没有偏置
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0, bias=False),
                # 对192通道进行BatchNorm
                nn.BatchNorm2d(192),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                
                # 第九层卷积层，输入192通道，输出10通道，卷积核大小为1，步幅为1，padding为0
                nn.Conv2d(192, 10, kernel_size=1, stride=1, padding=0),
                # ReLU激活函数
                nn.ReLU(inplace=True),
                # 平均池化层，池化核大小为8，步幅为1，padding为0
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
                )

    def forward(self, x):
        # 前向传播，通过分类器进行处理
        x = self.classifier(x)
        # 将特征图展平成一维向量，作为网络输出的logits
        logits = x.view(x.size(0), self.num_classes)
        # 使用softmax函数计算类概率分布
        probas = torch.softmax(logits, dim=1)
        return logits, probas

In [11]:
start_time = time.time()  # 记录开始时间
for epoch in range(NUM_EPOCHS):  # 循环遍历每个epoch
    
    model.train()  # 设置模型为训练模式
    
    for batch_idx, (features, targets) in enumerate(train_loader):  # 循环遍历每个批次
    
        ### 准备小批量数据
        features = features.to(DEVICE)  # 将特征转移到指定设备
        targets = targets.to(DEVICE)  # 将目标标签转移到指定设备
            
        ### 前向传播和反向传播
        logits, probas = model(features)  # 获取模型的输出
        cost = F.cross_entropy(logits, targets)  # 计算交叉熵损失
        optimizer.zero_grad()  # 清空梯度
        
        cost.backward()  # 反向传播计算梯度
        
        ### 更新模型参数
        optimizer.step()  # 使用优化器更新参数
        
        ### 日志记录
        if not batch_idx % 120:  # 每120个批次输出一次日志
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

    # 计算准确率时不需要构建计算图以进行反向传播
    with torch.set_grad_enabled(False):  # 禁用梯度计算
        train_acc = compute_accuracy(model, train_loader, device=DEVICE)  # 计算训练集准确率
        valid_acc = compute_accuracy(model, valid_loader, device=DEVICE)  # 计算验证集准确率
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')  # 输出准确率
    
    elapsed = (time.time() - start_time)/60  # 计算已经过去的时间（分钟）
    print(f'Time elapsed: {elapsed:.2f} min')  # 输出已用时间
  
elapsed = (time.time() - start_time)/60  # 计算总训练时间（分钟）
print(f'Total Training Time: {elapsed:.2f} min')  # 输出总训练时间

Epoch: 001/100 | Batch 000/192 | Cost: 0.9971
Epoch: 001/100 | Batch 120/192 | Cost: 0.8200
Epoch: 001/100 Train Acc.: 69.53% | Validation Acc.: 65.40%
Time elapsed: 1.81 min
Epoch: 002/100 | Batch 000/192 | Cost: 0.9075
Epoch: 002/100 | Batch 120/192 | Cost: 0.8703
Epoch: 002/100 Train Acc.: 67.59% | Validation Acc.: 64.80%
Time elapsed: 3.63 min
Epoch: 003/100 | Batch 000/192 | Cost: 0.9057
Epoch: 003/100 | Batch 120/192 | Cost: 0.9471
Epoch: 003/100 Train Acc.: 67.44% | Validation Acc.: 65.10%
Time elapsed: 5.44 min
Epoch: 004/100 | Batch 000/192 | Cost: 0.9677
Epoch: 004/100 | Batch 120/192 | Cost: 0.8854
Epoch: 004/100 Train Acc.: 69.69% | Validation Acc.: 66.70%
Time elapsed: 7.25 min
Epoch: 005/100 | Batch 000/192 | Cost: 0.9167
Epoch: 005/100 | Batch 120/192 | Cost: 0.8702
Epoch: 005/100 Train Acc.: 67.90% | Validation Acc.: 66.20%
Time elapsed: 9.07 min
Epoch: 006/100 | Batch 000/192 | Cost: 0.9741
Epoch: 006/100 | Batch 120/192 | Cost: 1.0208
Epoch: 006/100 Train Acc.: 70.09%

In [12]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 77.02%
